In [2]:
!pip install lightfm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 7.1 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for lightfm: filename=lightfm-1.17-cp310-cp310-linux_x86_64.whl size=447717 sha256=0966211e1a30a9fe857b589e655bdd5d892ce2205858f2438fe501d857720ea5
  Stored in directory: /root/.cache/pip/wheels/4f/9b/7e/0b256f2168511d8fa4dae4fae0200fdbd729eb424a912ad636
Successfully built lightfm


In [5]:
import pandas as pd
import numpy as np
from lightfm import LightFM
from lightfm.data import Dataset
from lightfm.evaluation import precision_at_k

In [6]:
tracks_df = pd.read_csv('/kaggle/input/tracks/cleaned_tracks_dataset.csv')
users_df = pd.read_csv('/kaggle/input/tracks/users_tracks_cleaned.csv')

In [7]:
dataset = Dataset()
dataset.fit(users_df['user_id'].unique(), tracks_df['track_id'].unique())

In [8]:
track_features = tracks_df[['track_id', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms']]
track_features = track_features.set_index('track_id')

track_features_binary = track_features.apply(lambda x: (x > x.median()).astype(int))

dataset.fit_partial(items=track_features_binary.index, item_features=track_features_binary.columns)

interactions, weights = dataset.build_interactions([(row['user_id'], track_id) for _, row in users_df.iterrows() for track_id in row[1:21].values])

track_features_matrix = dataset.build_item_features([(track_id, track_features_binary.loc[track_id].values) for track_id in track_features_binary.index])

model = LightFM(loss='warp')


In [9]:
model.fit(interactions, item_features=track_features_matrix, epochs=30)

In [12]:
def get_recommendations(user_id, top_n=3):
    user_index = np.where(users_df['user_id'] == user_id)[0][0]
    scores = model.predict([user_index] * len(tracks_df), np.arange(len(tracks_df)), item_features=track_features_matrix)
    top_indices = np.argsort(scores)[-top_n:][::-1]
    recommended_tracks = tracks_df.iloc[top_indices]['track_id'].values
    return recommended_tracks


In [13]:
recommendations_df = pd.DataFrame(columns=['user', 'rec_1', 'rec_2', 'rec_3'])

for user_id in users_df['user_id'].unique():
    rec_tracks = get_recommendations(user_id)
    recommendations_df.loc[len(recommendations_df)] = {
        'user': user_id,
        'rec_1': rec_tracks[0],
        'rec_2': rec_tracks[1],
        'rec_3': rec_tracks[2]
    }

recommendations_df.to_csv('user_recommendations_LightFM.csv', index=False)